In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기

In [9]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999 entries, 1 to 6999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  5423 non-null   float64
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6049 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Gender               6999 non-null   object 
 8   Discount_offered     3468 non-null   float64
 9   Weight_in_gms        6999 non-null   object 
 10  Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 656.2+ KB


# 이상치, 결측치 제거

In [18]:
# 공백문자가 포함된 컬럼 warehouse_block 수정
train = train.rename(columns = {'Warehouse_block ': 'Warehouse_block'})
test = test.rename(columns = {'Warehouse_block ': 'Warehouse_block'})

In [21]:
# 잘못 입력된 데이터 Shipzk, Flightzk, Roadzk 변경, 각 유니크 값 앞에 있는 공백문자 제거
train['Mode_of_Shipment'].unique()

array([' Ship', ' Flight', ' Road', '?', ' Shipzk', ' Flightzk',
       ' Roadzk'], dtype=object)

In [22]:
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Roadzk', 'Road')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Ship', 'Ship')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Flight', 'Flight')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Road', 'Road')

In [30]:
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Roadzk', 'Road')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Ship', 'Ship')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Flight', 'Flight')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Road', 'Road')

In [25]:
# 99로 잘못 입력된 유니크값 처리 평균, 중앙 최빈값 모두 3이므로 3으로 변경
train['Customer_rating'].unique()

array([ 2,  3,  1,  5,  4, 99])

In [26]:
train['Customer_rating']=train['Customer_rating'].replace(99, 3)

In [34]:
test['Customer_rating']=test['Customer_rating'].replace(99, 3)

In [28]:
# mediumm, loww, highh 유니크값 변경
train['Product_importance'].unique()

array(['low', 'medium', 'high', '?', 'mediumm', 'loww', 'highh'],
      dtype=object)

In [29]:
train['Product_importance'] = train['Product_importance'].replace('mediumm', 'medium')
train['Product_importance'] = train['Product_importance'].replace('loww', 'low')
train['Product_importance'] = train['Product_importance'].replace('highh', 'high')

In [35]:
test['Product_importance'] = test['Product_importance'].replace('mediumm', 'medium')
test['Product_importance'] = test['Product_importance'].replace('loww', 'low')
test['Product_importance'] = test['Product_importance'].replace('highh', 'high')

In [38]:
weight_question = train[train['Weight_in_gms']=='?']

In [39]:
weight_question

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,,
48,D,Ship,NaN,2,281,4.0,high,F,5.0,?,1
66,D,Ship,4.0,1,249,6.0,medium,M,NaN,?,1
68,A,?,3.0,2,213,6.0,medium,M,NaN,?,1
83,A,Ship,NaN,1,105,6.0,medium,M,62.0,?,1
91,C,Ship,NaN,2,189,6.0,low,M,NaN,?,0
...,...,...,...,...,...,...,...,...,...,...,...
6890,D,Ship,7.0,5,282,5.0,low,M,NaN,?,1
6915,A,Ship,5.0,4,157,4.0,low,M,NaN,?,1
6946,D,Ship,2.0,2,219,3.0,low,F,NaN,?,1


In [40]:
mask = train['Weight_in_gms'].isin(['?']) 
train[~mask]

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,,
1,C,Ship,2.0,2,135,3.0,low,M,NaN,1339,1
2,F,Flight,5.0,3,225,5.0,low,F,6.0,1082,0
3,F,Ship,3.0,1,229,3.0,low,M,NaN,4971,1
4,F,Ship,3.0,2,228,NaN,medium,M,2.0,5640,0
5,D,Flight,NaN,1,195,6.0,high,F,NaN,4944,1
...,...,...,...,...,...,...,...,...,...,...,...
6995,D,Ship,4.0,2,232,5.0,medium,F,4.0,1783,0
6996,F,Ship,3.0,5,228,2.0,medium,F,NaN,5208,0
6997,D,Ship,NaN,1,300,5.0,low,F,10.0,1787,0


# 시각화

# 전처리

# 스케일링

In [3]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
X_train_Scaled = scaler.transform(X_train)
X_test_Scaled = scaler.transform(X_test)

# 인코딩

In [ ]:
X_train = pd.get_dummies(X_train_Scaled)
X_test = pd.get_dummies(X_test_Scaled)

# 모델 선정 및 학습

In [ ]:
!pip install lightgbm

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# cv파라미터에 사용할 kfold값 설정
kfold = KFold(n_splits=10, random_state=123, shuffle=True)

In [ ]:
# lightgbm 모델
lgbm_model = LGBMClassifier(n_estimators=1000, learning_rate=0.01, random_state=123)
lgbm_model.fit(X_train, y_train)

In [ ]:
# 정확도 교차검증
acc_result = cross_val_score(lgbm_model,
                            X_train,
                            y_train,
                            cv=kfold
                            )
print(acc_result.mean())

In [ ]:
# f1스코어 교차검증
f1_result = cross_val_score(lgbm_model,
                           X_train,
                           y_train,
                           cv=kfold,
                           scoring='f1'
                           )
print(f1_result.mean())

In [ ]:
# xgboost 모델
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.01, random_state=123)
xgb_model.fit(X_train, y_train)

In [ ]:
# 정확도 교차검증
acc_result = cross_val_score(xgb_model,
                            X_train,
                            y_train,
                            cv=kfold
                            )
print(acc_result.mean())

In [ ]:
# f1스코어 교차검증
f1_result = cross_val_score(xgb_model,
                           X_train,
                           y_train,
                           cv=kfold,
                           scoring='f1'
                           )
print(f1_result.mean())

In [ ]:
# randomforest 모델

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=11)
rf_model.fit(X_train, y_train)

In [ ]:
# 정확도 교차검증
acc_result = cross_val_score(rf_model,
                            X_train,
                            y_train,
                            cv=kfold
                            )
print(acc_result.mean())

In [ ]:
# f1스코어 교차검증
f1_result = cross_val_score(rf_model,
                           X_train,
                           y_train,
                           cv=kfold,
                           scoring='f1'
                           )
print(f1_result.mean())